# 医学多模态 CLIP 教学（新手友好版）

欢迎！这个 Notebook 会一步一步带你做一个 **可以跑通的、基于 CLIP 的医学多模态（图像 + 文本）分析** 小教程。

你将学到：
- 数据如何准备、如何存放（适用于自己的医学数据）
- 如何加载预训练 CLIP
- 如何进行微调（fine-tune）
- 如何评估（简单准确率/检索 Top-1）
- 如何保存与推理

> **定位**：面向完全新手，代码注释非常清楚，每一步在做什么都写明白。


## 0. 安装依赖（只需要做一次）

在终端执行：
```bash
# 建议在虚拟环境中安装
pip install torch torchvision --index-url https://download.pytorch.org/whl/cpu
pip install transformers datasets pillow scikit-learn matplotlib
```

> 如果你有 GPU，可以把 PyTorch 安装为 GPU 版本（速度更快）。


## 1. 数据准备（非常关键）

我们需要 **图像 + 文本** 的配对数据。

### 1.1 推荐的数据目录结构
```
my_med_data/
  images/
    0001.png
    0002.png
    ...
  metadata.csv
```

### 1.2 metadata.csv 格式
至少包含两列：
- `image_path`：相对路径，例如 `images/0001.png`
- `report`：与图像对应的文本描述（比如放射科报告）

示例：
```csv
image_path,report
images/0001.png,"胸片显示双肺纹理增粗..."
images/0002.png,"心影增大，考虑心衰..."
```

> **你的真实数据**：就按这个格式整理即可。


## 2. （可选）生成一个可跑通的玩具数据集

如果你暂时没有真实数据，先用下面的代码生成一个 **小型示例数据**，这样你可以快速跑通整个流程。

> 这一步只用于演示，不代表真实医学结果。


In [ ]:
import os
from pathlib import Path
from PIL import Image, ImageDraw
import csv

# 1) 设置数据保存路径
root_dir = Path("toy_med_data")
img_dir = root_dir / "images"
img_dir.mkdir(parents=True, exist_ok=True)

# 2) 准备一些简单的“报告文本”
reports = [
    "胸片显示双肺纹理增粗，考虑轻度炎症。",
    "心影略大，建议结合临床进一步检查。",
    "右下肺可见片状阴影，考虑感染。",
    "左肺透亮度稍减，可能有轻度积液。",
    "肺野清晰，未见明显异常。",
]

# 3) 生成简易图片（仅用于示例）
for i, text in enumerate(reports, start=1):
    img = Image.new("RGB", (256, 256), color=(255, 255, 255))
    draw = ImageDraw.Draw(img)
    draw.rectangle([40, 40, 216, 216], outline=(0, 0, 0), width=3)
    draw.text((50, 110), f"ID {i}", fill=(0, 0, 0))
    img_path = img_dir / f"{i:04d}.png"
    img.save(img_path)

# 4) 写入 metadata.csv
csv_path = root_dir / "metadata.csv"
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["image_path", "report"])
    for i, text in enumerate(reports, start=1):
        writer.writerow([f"images/{i:04d}.png", text])

print("玩具数据集已生成：", root_dir)


## 3. 读取数据并构建 Dataset

这里我们自己写一个 `Dataset` 类，完成以下工作：
- 读取 `metadata.csv`
- 加载图像
- 返回图像和文本

**这一步是训练前最重要的输入准备。**


In [ ]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from PIL import Image

class MedImageTextDataset(Dataset):
    def __init__(self, root_dir, csv_file):
        self.root_dir = Path(root_dir)
        self.df = pd.read_csv(self.root_dir / csv_file)

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image_path = self.root_dir / row["image_path"]
        text = row["report"]
        image = Image.open(image_path).convert("RGB")
        return image, text

# 这里使用玩具数据（如果你有自己的数据，改成你的路径）
dataset = MedImageTextDataset("toy_med_data", "metadata.csv")
print("样本数：", len(dataset))
print("示例：", dataset[0])


## 4. 加载 CLIP 模型与处理器

CLIP 需要同时处理图像和文本，所以我们使用 `CLIPProcessor`。

**重要**：这里会下载预训练模型（需要网络）。


In [ ]:
import torch
from transformers import CLIPProcessor, CLIPModel

# 选择一个轻量级模型，适合新手跑通
model_name = "openai/clip-vit-base-patch32"

processor = CLIPProcessor.from_pretrained(model_name)
model = CLIPModel.from_pretrained(model_name)

# 如果有 GPU 可以加速
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

print("设备：", device)


## 4.1 运行前的快速自检（确保流程能跑通）

这一步做一个 **最小化的前向计算**，确认：
- 数据可以被处理器正确读取
- 模型可以正常前向推理并返回 loss

> 如果这一步报错，通常是环境依赖或数据路径问题，请先修复。


In [ ]:
import torch

# 取一个小 batch 做自检
sample_images = [dataset[0][0], dataset[1][0]]
sample_texts = [dataset[0][1], dataset[1][1]]

with torch.no_grad():
    sample_inputs = processor(text=sample_texts, images=sample_images, return_tensors="pt", padding=True)
    sample_inputs = {k: v.to(device) for k, v in sample_inputs.items()}
    sample_outputs = model(**sample_inputs, return_loss=True)

print("自检 loss:", sample_outputs.loss.item())


## 5. 构建 DataLoader 和训练循环

CLIP 的训练目标是：**图像与文本在同一向量空间里靠近**。

我们使用 `model(**inputs, return_loss=True)` 直接得到对比学习 loss。


In [ ]:
def collate_fn(batch):
    images, texts = zip(*batch)
    inputs = processor(text=list(texts), images=list(images), return_tensors="pt", padding=True)
    return inputs

loader = DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

model.train()
for epoch in range(2):  # 演示用 2 个 epoch
    total_loss = 0
    for step, batch in enumerate(loader):
        # 把输入放到设备上
        batch = {k: v.to(device) for k, v in batch.items()}

        # 计算 loss
        outputs = model(**batch, return_loss=True)
        loss = outputs.loss

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(loader)
    print(f"Epoch {epoch+1} | loss = {avg_loss:.4f}")


## 6. 简单评估：图文匹配 Top-1

这里做一个最直观的评估：
- 对于每张图，找最相似的文本
- 如果找到的文本就是它原本配对的文本，则算对

> 这是一个简单演示，真实场景可计算更专业指标（如 Recall@K）。


In [ ]:
import torch.nn.functional as F

model.eval()

# 准备全部图像和文本
all_images = []
all_texts = []
for img, txt in dataset:
    all_images.append(img)
    all_texts.append(txt)

with torch.no_grad():
    inputs = processor(text=all_texts, images=all_images, return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)

    image_embeds = outputs.image_embeds
    text_embeds = outputs.text_embeds

    # 归一化后计算相似度
    image_embeds = F.normalize(image_embeds, p=2, dim=-1)
    text_embeds = F.normalize(text_embeds, p=2, dim=-1)

    similarity = image_embeds @ text_embeds.T  # (N, N)

# Top-1 准确率
correct = 0
for i in range(similarity.size(0)):
    pred = similarity[i].argmax().item()
    if pred == i:
        correct += 1

acc = correct / similarity.size(0)
print(f"Top-1 图文匹配准确率: {acc:.2f}")


## 7. 推理示例：给一张图找最相关文本

下面展示 **图像检索文本** 的效果。


In [ ]:
# 取第一张图片进行检索
query_image = all_images[0]

with torch.no_grad():
    inputs = processor(text=all_texts, images=[query_image], return_tensors="pt", padding=True)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model(**inputs)

    image_embed = F.normalize(outputs.image_embeds, p=2, dim=-1)
    text_embeds = F.normalize(outputs.text_embeds, p=2, dim=-1)
    scores = (image_embed @ text_embeds.T).squeeze(0)

best_idx = scores.argmax().item()
print("最相关文本：", all_texts[best_idx])


## 8. 如何用自己的真实数据（详细指导）

### 8.1 准备自己的数据
1. 把医学图像放到 `my_med_data/images/`
2. 创建 `my_med_data/metadata.csv`，至少包含 `image_path` 和 `report`

### 8.2 训练前需要确认的点
- 图像格式是否统一（png/jpg）
- 文本是否清洗过（去掉隐私信息）
- 数据量是否足够（越多越好）

### 8.3 训练策略建议
- **小数据集**：建议只微调文本/图像编码器后几层，避免过拟合
- **中大数据集**：可以全量微调
- 学习率建议从 `1e-5 ~ 5e-5` 之间尝试
- batch size 视显存而定

### 8.4 你的数据接入方式（改这一行）
```python
# 把 toy_med_data 替换成你的目录
# dataset = MedImageTextDataset("my_med_data", "metadata.csv")
```


## 9. 如何做“预训练” vs “微调”

### 9.1 预训练（从零开始）
- 需要大量数据（几十万到百万级别）
- 训练时间长，算力消耗大
- 实际工作中，通常 **不建议新手从零预训练**

### 9.2 微调（推荐）
- 基于现成的 CLIP 权重
- 使用你自己的医学数据做少量训练
- 成本低，效果提升明显

> 本教程就是微调的流程。


## 10. 保存与加载模型

训练后记得保存模型，这样下次可以直接加载。


In [ ]:
# 保存
save_dir = "clip_med_finetuned"
model.save_pretrained(save_dir)
processor.save_pretrained(save_dir)
print("模型已保存到：", save_dir)

# 加载
# model = CLIPModel.from_pretrained(save_dir)
# processor = CLIPProcessor.from_pretrained(save_dir)


## 11. 常见问题（新手最容易卡住的地方）

1. **下载模型失败怎么办？**
   - 需要联网，或提前把模型下载到本地。

2. **CPU 太慢怎么办？**
   - 尝试使用 GPU，或减少 batch size。

3. **报错 “CUDA out of memory”**
   - 减少 batch size 或使用更小模型。

4. **如何加入更多医学任务？**
   - 可以在 CLIP embedding 后加一个分类头，用于疾病分类。

---

🎉 恭喜你完成了一个完整的 CLIP 多模态医学教程！
